## Imports

In [13]:
import numpy as np
import linecache

## Aggregating postcodes by alphabetical prefix

In [4]:
customer_id_og = linecache.getline(r"CaseStudyData.txt", 17).split(" ")[2:-1]
customer_id_og = [id.strip('"') for id in customer_id_og]

candidate_id_og = linecache.getline(r"CaseStudyData.txt", 27).split(" ")[2:-1]
candidate_id_og = [id.strip('"') for id in candidate_id_og]

In [5]:
def aggregate_postcodes(list):
    
    # Extract unique prefixes
    unique_prefixes = np.unique([postcode[:2] if len(postcode) >= 2 and postcode[1].isalpha() else postcode[0] for postcode in list])

    # Create array with prefixes
    aggregated_values = np.array([postcode[:2] if len(postcode) >= 2 and postcode[1].isalpha() else postcode[0] for postcode in list])

    return unique_prefixes, aggregated_values


# Display the aggregated values
# for i, prefix in enumerate(unique_prefixes):
#     count = np.count_nonzero(aggregated_values == prefix)
#     print(f"Aggregate for prefix {prefix}: {count}")

customer_id = aggregate_postcodes(customer_id_og)
candidate_id = aggregate_postcodes(candidate_id_og)

### I think the CustomerID and CandidateID vectors are all the same.

In [46]:
def process_aggregated_values(postcode_list, value_list, operation='average'):
    unique_prefixes, aggregated_values = aggregate_postcodes(postcode_list)
    
    values_dict = {prefix: [] for prefix in unique_prefixes}

    for prefix, value in zip(aggregated_values, value_list):
        values_dict[prefix].append(value)

    if operation == 'average':
        result_array = np.array([np.mean(values_dict[prefix]) for prefix in unique_prefixes])
    elif operation == 'sum':
        result_array = np.array([np.sum(values_dict[prefix]) for prefix in unique_prefixes])
    elif operation == 'maximum':
        result_array = np.array([np.max(values_dict[prefix]) for prefix in unique_prefixes])
    else:
        raise ValueError("Invalid operation. Please choose 'average', 'sum', or 'maximum'.")

    return result_array

## Aggregating coordinates

In [44]:
customer_easting = linecache.getline(r"CaseStudyData.txt", 20).split(" ")[2:-1]
customer_easting = [eval(coord) for coord in customer_easting]
customer_easting_avg = process_aggregated_values(customer_id_og, customer_easting)

customer_northing = linecache.getline(r"CaseStudyData.txt", 21).split(" ")[2:-1]
customer_northing = [eval(coord) for coord in customer_northing]
customer_northing_avg = process_aggregated_values(customer_id_og, customer_northing)

candidate_easting = linecache.getline(r"CaseStudyData.txt", 30).split(" ")[2:-1]
candidate_easting = [eval(coord) for coord in candidate_easting]
candidate_easting_avg = process_aggregated_values(candidate_id_og, candidate_easting)

candidate_northing = linecache.getline(r"CaseStudyData.txt", 31).split(" ")[2:-1]
candidate_northing = [eval(coord) for coord in candidate_northing]
candidate_northing_avg = process_aggregated_values(candidate_id_og, candidate_northing)

## Aggregating  warehouse costs and capacities

In [36]:
def process_costs_capacities(line_start, line_stop):
    file = open(r"CaseStudyData.txt", "r")
    costs_og = file.readlines()[line_start-1:line_stop]
    costs_og = list(map(lambda s: s.strip(), costs_og))

    costs_og = [costs_og[line].split(" ") for line in range(len(costs_og))]
    costs = []
    for line in costs_og:
        for i in range(len(line)):
            costs.append(line[i].strip('[]'))
            
    costs = costs[2:]        
    costs = [eval(cost) for cost in costs]
    return costs

In [52]:
setup_costs = process_costs_capacities(62, 105)
operating_costs = process_costs_capacities(108, 146)
wh_capacities = process_costs_capacities(149, 193)

setup_costs_agg = process_aggregated_values(customer_id_og, setup_costs, operation = "maximum")
operating_costs_agg = process_aggregated_values(customer_id_og, operating_costs, operation = "maximum")
wh_capacities_agg = process_aggregated_values(customer_id_og, wh_capacities, operation = "maximum")